# Decision Trees


## Objectives
*   Develop a classification model using Decision Tree Algorithm


Will use this classification algorithm to build a model from the historical data of patients, and their response to different medications. Then, will use the trained decision tree to predict the class of a unknown patient, or to find a proper drug for a new patient.


<h1>Table of contents</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#about_dataset">About the dataset</a></li>
        <li><a href="#downloading_data">Downloading the Data</a></li>
        <li><a href="#pre-processing">Pre-processing</a></li>
        <li><a href="#setting_up_tree">Setting up the Decision Tree</a></li>
        <li><a href="#modeling">Modeling</a></li>
        <li><a href="#prediction">Prediction</a></li>
        <li><a href="#evaluation">Evaluation</a></li>
        <li><a href="#visualization">Visualization</a></li>
    </ol>
</div>
<br>
<hr>


<div id="about_dataset">
    <h2>About the dataset</h2>
    Imagine that you are a medical researcher compiling data for a study. You have collected data about a set of patients, all of whom suffered from the same illness. During their course of treatment, each patient responded to one of 5 medications, Drug A, Drug B, Drug c, Drug x and y. 
    <br>
    <br>
    Part of your job is to build a model to find out which drug might be appropriate for a future patient with the same illness. The features of this dataset are Age, Sex, Blood Pressure, and the Cholesterol of the patients, and the target is the drug that each patient responded to.
    <br>
    <br>
    It is a sample of multiclass classifier, and you can use the training part of the dataset 
    to build a decision tree, and then use it to predict the class of a unknown patient, or to prescribe a drug to a new patient.
</div>

In [62]:
import numpy as np 
import pandas as pd

### **Reading the Data**

Now, read the data using pandas dataframe:


In [25]:
df = pd.read_csv("drug200.csv", delimiter=",")
df.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,drugY


In [26]:
#Shape of Data
df.shape

(200, 6)

<div href="pre-processing">
    <h2>Pre-processing</h2>
</div>


Using <b>my_data</b> as the Drug.csv data read by pandas, declare the following variables: <br>

<ul>
    <li> <b> X </b> as the <b> Feature Matrix </b> (data of my_data) </li>
    <li> <b> y </b> as the <b> response vector </b> (target) </li>
</ul>


Remove the column containing the target name since it doesn't contain numeric values.


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          200 non-null    int64  
 1   Sex          200 non-null    object 
 2   BP           200 non-null    object 
 3   Cholesterol  200 non-null    object 
 4   Na_to_K      200 non-null    float64
 5   Drug         200 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 9.5+ KB


As you may figure out, some features in this dataset are categorical, such as **Sex** or **BP**. Unfortunately, Sklearn Decision Trees does not handle categorical variables. We can still convert these features to numerical values using **pandas.get_dummies()**
to convert the categorical variable into dummy/indicator variables.


In [80]:
df.BP.unique()

array(['HIGH', 'LOW', 'NORMAL'], dtype=object)

In [81]:
df.Cholesterol.unique()

array(['HIGH', 'NORMAL'], dtype=object)

In [82]:
cat_feats = ['Drug', 'Sex', 'BP','Cholesterol']

In [83]:
final_data = pd.get_dummies(df,columns=cat_feats,drop_first=True)

In [116]:
final_data.head()

,Age,Na_to_K,Drug_drugB,Drug_drugC,Drug_drugX,Drug_drugY,Sex_M,BP_LOW,BP_NORMAL,Cholesterol_NORMAL
0,23,25.355,0,0,0,1,0,0,0,0
1,47,13.093,0,1,0,0,1,1,0,0
2,47,10.114,0,1,0,0,1,1,0,0
3,28,7.798,0,0,1,0,0,0,1,0
4,61,18.043,0,0,0,1,0,1,0,0


In [71]:
# <!-- **Interpretation of Dummy variables**
# <li>Sex: Male = 1, Female = 0, </li>
# <li>About the dataset</li> -->   

<hr>

<div id="setting_up_tree">
    <h2>Setting up the Decision Tree</h2>
    We will be using <b>train/test split</b> on our <b>decision tree</b>. Let's import <b>train_test_split</b> from <b>sklearn.cross_validation</b>.
</div>


In [87]:
X = final_data.drop(['Drug_drugB', 'Drug_drugC', 'Drug_drugX', 'Drug_drugY'], axis=1)
y = final_data[['Drug_drugB', 'Drug_drugC', 'Drug_drugX', 'Drug_drugY']]

In [90]:
y.head()

,Drug_drugB,Drug_drugC,Drug_drugX,Drug_drugY
0,0,0,0,1
1,0,1,0,0
2,0,1,0,0
3,0,0,1,0
4,0,0,0,1


In [91]:
X.head()

,Age,Na_to_K,Sex_M,BP_LOW,BP_NORMAL,Cholesterol_NORMAL
0,23,25.355,0,0,0,0
1,47,13.093,1,1,0,0
2,47,10.114,1,1,0,0
3,28,7.798,0,0,1,0
4,61,18.043,0,1,0,0


In [92]:
from sklearn.model_selection import train_test_split

Now <b> train_test_split </b> will return 4 different parameters. We will name them:<br>
X_trainset, X_testset, y_trainset, y_testset <br> <br>
The <b> train_test_split </b> will need the parameters: <br>
X, y, test_size=0.3, and random_state=3. <br> <br>
The <b>X</b> and <b>y</b> are the arrays required before the split, the <b>test_size</b> represents the ratio of the testing dataset, and the <b>random_state</b> ensures that we obtain the same splits.


In [93]:
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.3)

<h3>Practice</h3>
Print the shape of X_trainset and y_trainset. Ensure that the dimensions match.


In [94]:
print('Shape of X training set {}'.format(X_trainset.shape),'&',' Size of Y training set {}'.format(y_trainset.shape))

Shape of X training set (140, 6) &  Size of Y training set (140, 4)


Print the shape of X_testset and y_testset. Ensure that the dimensions match.


In [95]:
print('Shape of X test set {}'.format(X_testset.shape),'&',' Size of Y test set {}'.format(y_testset.shape))

Shape of X test set (60, 6) &  Size of Y test set (60, 4)


<hr>

<div id="modeling">
    <h2>Modeling</h2>
    We will first create an instance of the <b>DecisionTreeClassifier</b> called <b>drugTree</b>.<br>
    Inside of the classifier, specify <i> criterion="entropy" </i> so we can see the information gain of each node.
</div>


In [99]:
from sklearn.tree import DecisionTreeClassifier

In [100]:
drugTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4) #criterion="entropy", max_depth = 4
drugTree # it shows the default parameters

DecisionTreeClassifier(criterion='entropy', max_depth=4)

Next, we will fit the data with the training feature matrix <b> X_trainset </b> and training  response vector <b> y_trainset </b>


In [101]:
drugTree.fit(X_trainset,y_trainset)

DecisionTreeClassifier(criterion='entropy', max_depth=4)

<hr>

<div id="prediction">
    <h2>Prediction</h2>
    Let's make some <b>predictions</b> on the testing dataset and store it into a variable called <b>predTree</b>.
</div>


In [102]:
predTree = drugTree.predict(X_testset)

You can print out <b>predTree</b> and <b>y_testset</b> if you want to visually compare the predictions to the actual values.


In [103]:
print (predTree [0:5])
print (y_testset [0:5])


[[0 0 1 0]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 0 0]]
     Drug_drugB  Drug_drugC  Drug_drugX  Drug_drugY
67            0           0           1           0
159           0           0           1           0
84            0           1           0           0
4             0           0           0           1
110           0           0           0           0


<hr>

<div id="evaluation">
    <h2>Evaluation</h2>
    Next, let's import <b>metrics</b> from sklearn and check the accuracy of our model.
</div>


In [104]:
from sklearn import metrics
import matplotlib.pyplot as plt
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_testset, predTree))

DecisionTrees's Accuracy:  1.0


**Accuracy classification score** computes subset accuracy: the set of labels predicted for a sample must exactly match the corresponding set of labels in y_true.

In multilabel classification, the function returns the subset accuracy. If the entire set of predicted labels for a sample strictly match with the true set of labels, then the subset accuracy is 1.0; otherwise it is 0.0.


In [112]:
from sklearn.metrics import classification_report

In [114]:
print(classification_report(y_testset,predTree))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00        16
           3       1.00      1.00      1.00        25

   micro avg       1.00      1.00      1.00        55
   macro avg       1.00      1.00      1.00        55
weighted avg       1.00      1.00      1.00        55
 samples avg       0.92      0.92      0.92        55



## Training the Random Forest model

Now its time to train our model!

**Create an instance of the RandomForestClassifier class and fit it to our training data from the previous step.**

In [119]:
from sklearn.ensemble import RandomForestClassifier

rc = RandomForestClassifier(n_estimators=800)
rc.fit(X_trainset,y_trainset)

RandomForestClassifier(n_estimators=800)

## Predictions and Evaluation

Let's predict off the y_test values and evaluate our model.

In [120]:
pred = rc.predict(X_testset)

In [124]:
from sklearn import metrics
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_testset, pred))

DecisionTrees's Accuracy:  0.9833333333333333


**Now we create a classification report from the results**

In [123]:
from sklearn.metrics import classification_report
print(classification_report(y_testset,pred))

              precision    recall  f1-score   support

           0       1.00      0.88      0.93         8
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00        16
           3       1.00      1.00      1.00        25

   micro avg       1.00      0.98      0.99        55
   macro avg       1.00      0.97      0.98        55
weighted avg       1.00      0.98      0.99        55
 samples avg       0.90      0.90      0.90        55



In [115]:
# Notice: You might need to uncomment and install the pydotplus and graphviz libraries if you have not installed these before
#!conda install -c conda-forge pydotplus -y
#!conda install -c conda-forge python-graphviz -y